In [1]:
# 显示上传的数据集
!ls /home/aistudio/data/data113944/

steel_bug_detect.zip


In [2]:
!nvidia-smi

Tue Jul 29 08:05:00 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:A1:00.0 Off |                    0 |
| N/A   47C    P0   114W / 300W |      0MiB / 16384MiB |     28%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!unzip /home/aistudio/data/data113944/steel_bug_detect.zip

In [3]:
# paddle环境准备
#!git clone https://gitee.com/paddlepaddle/PaddleDetection.git
%cd PaddleDetection
# 安装其他依赖
! pip install paddledet==2.0.1 -i https://mirror.baidu.com/pypi/simple
! pip install -r requirements.txt

/home/aistudio/PaddleDetection
Looking in indexes: https://mirror.baidu.com/pypi/simple, https://mirrors.aliyun.com/pypi/simple/
  Using cached https://mirrors.aliyun.com/pypi/packages/e4/d5/7b5175c8ce6c45c59ff5dd161083c7c9503f501a8b3ef4c3984fdee83556/paddledet-2.0.1-py3-none-any.whl (307 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/bb/bd/dc7da80c95c920ee2b575e64901b5962ca4a1271b5f3cf6c27242aa0aafc/typeguard-4.1.2-py3-none-any.whl (33 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/c4/fb/ea621e0a19733e01fe4005d46087d383693c0f4a8f824b47d8d4122c87e0/terminaltables-3.1.10-py2.py3-none-any.whl (15 kB)
  Using cached https://mirrors.aliyun.com/pypi/packages/b1/ee/834c24687d14123d7d94a4716154148d3710454dd36a9a8d0df8d15fe999/shapely-2.0.7-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.4 MB)
  Using cached https://mirrors.aliyun.com/pypi/packages/27/37/9a47974f4176d10d5304ead7f4c1ddf6e4148405d4e4c5414c4c68967295/pycocotools-2.0.7-cp37-cp37m-manylinux_2_

In [5]:
%cd PaddleDetection/
!pwd

/home/aistudio/PaddleDetection
/home/aistudio/PaddleDetection


In [4]:
!pwd
%cd ..

/home/aistudio/PaddleDetection
/home/aistudio


In [6]:
# 使用train.py进行训练
!python ./tools/train.py -c ../bug_detect/yolov3_mobilenet_v1_270e_voc.yml --eval --use_vdl=True

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W0729 08:05:57.148104   131 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 10.1
W0729 08:05:57.152534   131 device_context.cc:422] device: 0, cuDNN Version: 7.6.
[07/29 08:06:01] ppdet.utils.checkpoint INFO: Finish loading model weights: /home/aistudio/.cache/paddle/weights/MobileNetV1_pretrained.pdparams
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To sil

In [7]:
# 评估
!python ./tools/eval.py -c ../bug_detect/yolov3_mobilenet_v1_270e_voc.yml -o weights=./output/yolov3_mobilenet_v1_270e_voc/best_model.pdparams


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:
W0729 08:07:36.036557   640 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 12.0, Runtime API Version: 10.1
W0729 08:07:36.040022   640 device_context.cc:422] device: 0, cuDNN Version: 7.6.
[07/29 08:07:39] ppdet.utils.checkpoint INFO: The shape [18] in pretrained weight yolo_head.yolo_output.0.bias is unmatched with the shape [33] in model yolo_head.yolo_output.0.bias. And the weight yolo_head.yolo_output.0.bias will not be loaded
[07/29 08:07:39] ppdet.utils.checkpoint INFO: The shape [18, 1024, 1, 1] in pretrai

In [ ]:
import os
import pandas as pd

result = pd.DataFrame(columns=['image_id', 'bbox', 'category_id', 'confidence'])
result_map = {'crazing':0,'inclusion':1, 'pitted_surface':2, 'scratches':3, 'patches':4, 'rolled-in_scale':5}
# 找到你对应的文件目录
file_path = '../steel_bug_detect/test/yolov3_mobilenet49/'
#file_path = '../steel_bug_detect/test/faster_rcnnbest/'

def work(temp):
    global result
    with open(file_path + temp, 'r') as file:
        lines = file.readlines()
    #result.split(' ')

    image_id = temp.replace('.jpg.txt', '')
    image_id = temp.replace('.txt', '')

    image_id
    #print('lines=', lines)
    for line in lines:
        #xmin, ymin, xmax, ymax, category_id, confidence, _ = line.split(' ')
        line = line.replace('\n', '')
        #print(line)
        category, confidence, xmin, ymin, w, h = line.split(' ')
        xmin, ymin, w, h = float(xmin), float(ymin), float(w), float(h)
        xmax = xmin + w
        ymax = ymin + h
        category_id = result_map[category]
        xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
        bbox = [xmin, ymin, xmax, ymax]
        temp_result = {}
        temp_result['image_id'] = image_id
        temp_result['bbox'] = bbox
        temp_result['category_id'] = category_id
        temp_result['confidence'] = confidence
        result = result.append(temp_result, ignore_index=True)

    #print(xmin, ymin, xmax, ymax, category_id, confidence)
for temp in os.listdir(file_path):
    if temp[-4:] == '.txt':
        work(temp)
result = result.sort_values(by=['image_id'], ascending=True)
result.to_csv('./baseline_ppyolov2-874.csv', index=False)
#result.to_csv('./baseline_faster_rcnn-best.csv', index=False)
result

,image_id,bbox,category_id,confidence
27,1404,"[0, 0, 186, 159]",2,0.6203819513320923
72,1408,"[0, 0, 186, 158]",2,0.5877552032470703
22,1413,"[0, 0, 191, 163]",2,0.6269048452377319
63,1421,"[104, 3, 174, 106]",4,0.6326373815536499
39,1424,"[136, 75, 190, 123]",4,0.6665996313095093
...,...,...,...,...
53,1780,"[0, 21, 55, 93]",4,0.7472273707389832
54,1780,"[0, 101, 62, 199]",4,0.5921993851661682
34,1789,"[101, 17, 111, 102]",3,0.540276825428009
23,1790,"[49, 94, 105, 172]",4,0.748997688293457


In [ ]:
result['confidence'] = result['confidence'].astype('float64')
result.loc[result['confidence']>=0.9, 'confidence'] = 1
result.to_csv('./result-submittion.csv', index=False)
result

,image_id,bbox,category_id,confidence
27,1404,"[0, 0, 186, 159]",2,0.620382
72,1408,"[0, 0, 186, 158]",2,0.587755
22,1413,"[0, 0, 191, 163]",2,0.626905
63,1421,"[104, 3, 174, 106]",4,0.632637
39,1424,"[136, 75, 190, 123]",4,0.666600
...,...,...,...,...
53,1780,"[0, 21, 55, 93]",4,0.747227
54,1780,"[0, 101, 62, 199]",4,0.592199
34,1789,"[101, 17, 111, 102]",3,0.540277
23,1790,"[49, 94, 105, 172]",4,0.748998


In [ ]:
# 预测
#!python ./tools/infer.py -c ../steel_bug_detect/train/yolov3_mobilenet_v1_270e_voc.yml -o weights=./output/yolov3_mobilenet_v1_270e_voc/best_model.pdparams --infer_dir=../steel_bug_detect/test/IMAGES --save_txt True

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the